In [1]:
from sklearn.ensemble import RandomForestClassifier
import os
import json
import pickle
from collections import defaultdict
from collections import Counter
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
with open('mappings/RF/data_dict.pkl', 'rb') as fp:
    data_dict = pickle.load(fp)

In [3]:
data = np.array([v for k,v in data_dict.items()])
data[:3]

array([[1.97822984e+00, 2.52062427e-01, 8.72504887e+01, 5.20000000e+01,
        1.15328580e+07, 2.21785731e+05, 4.70000000e+01, 1.00000000e+00],
       [1.98251249e+00, 2.25769163e-01, 2.63111262e+01, 3.90000000e+01,
        1.16681800e+07, 2.99184103e+05, 2.30000000e+01, 1.00000000e+00],
       [1.99500357e+00, 5.28044397e-01, 5.88093103e+00, 6.40000000e+01,
        1.40338890e+07, 2.19279516e+05, 5.10000000e+01, 1.00000000e+00]])

In [4]:
m = [x[-1] > 1 for x in data]
data = data[m]

In [5]:
#l1, l2, derived diversity, number_of_tracks  , total_duration , avg_song_length , num albums
x = data[:, :-1]
y = data[:,-1]
print(x[:2], y[:2], data[:2])

[[1.98643826e+00 2.46500032e-01 1.43684248e+01 1.70000000e+01
  4.24636700e+06 2.49786294e+05 1.60000000e+01]
 [1.97287652e+00 2.66156527e-01 1.86384247e+02 7.20000000e+01
  1.64029660e+07 2.27818972e+05 6.00000000e+01]] [2. 2.] [[1.98643826e+00 2.46500032e-01 1.43684248e+01 1.70000000e+01
  4.24636700e+06 2.49786294e+05 1.60000000e+01 2.00000000e+00]
 [1.97287652e+00 2.66156527e-01 1.86384247e+02 7.20000000e+01
  1.64029660e+07 2.27818972e+05 6.00000000e+01 2.00000000e+00]]


In [6]:
clf = RandomForestClassifier(n_estimators=30, verbose=3)
clf.fit(x, y)

building tree 1 of 30


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


building tree 2 of 30


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s remaining:    0.0s


building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   28.0s finished


RandomForestClassifier(n_estimators=30, verbose=3)

In [7]:
clf.score(x,y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    5.0s finished


0.9975739215034297

In [8]:
clf.feature_importances_

array([0.13075285, 0.16705722, 0.16892886, 0.09682199, 0.16319506,
       0.16521747, 0.10802655])

In [9]:
#l1, l2, derived diversity, number_of_tracks  , total_duration , avg_song_length , num albums


In [13]:
clf2 = RandomForestClassifier(n_estimators=30, verbose=3)
# clf2.fit(x, np.log(y))

In [15]:
y2 = np.ceil(np.log(y))
clf2.fit(x,y2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 30


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 30


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    6.9s finished


RandomForestClassifier(n_estimators=30, verbose=3)

In [17]:
clf2.score(x,y2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.8s finished


0.9982019925862982

In [18]:
clf2.feature_importances_

array([0.1289989 , 0.16867627, 0.1704031 , 0.09495595, 0.16486963,
       0.16645455, 0.10564159])